In [10]:
def xml_cleaner(line):
    """
    return plain text from a line of XML
    """
    while '<' in line:
        line = line.strip()
        #print("loop 1",end=' ')
        s = line.find('<')
        e = line.find('>') + 1
        try:
            while '<' == line[e]:
                #print("loop 2",end=' ')
                e = line[e:len(line)].find('>') + e +1
                #print(f'{e}')
        except IndexError:
            pass
        to_rep = line[s:e]
        #print(len(line),len(to_rep), sep='\n')
        if len(line)==len(to_rep):
            return ''
        #print(line,to_rep, sep='\n')
        line = line.replace(to_rep, ' ').strip()
    while '  ' in line:
        line = line.replace('  ',' ')
    while '- ' in line:
        line = line.replace('- ','\n').strip()
    return line

In [11]:
fin = open("/home/smoxy/Scrivania/Naldi/AIMLEA/AIMLEA4Suburra/RAW/SuburraS01E01.xml", encoding='UTF-8')
#c = 0
for line in fin:
    n_line = xml_cleaner(line)
    if len(n_line)>0:
        #print(line, end='')
        print(n_line)
    #c += 1
    #if c==20:
    #    print("finish")
    #    break
    

[radio] Oggi, dopo sette anni di mandato,
il sindaco di Roma ha presentato le proprie dimissioni
che saranno effettive entro 20 giorni.
[radio] Nel suo discorso di addio all'ultima riunione di Consiglio,
ha ribadito come la sua scelta sia stata molto sofferta, ma ponderata.
[radio] Il sindaco uscente ha ricordato
che il rispetto per gli elettori e per i cittadini di Roma
è sempre stato messo al primo posto.
[radio] Adesso si prospetta uno scenario di forte indecisione
per tutti gli schieramenti politici,
per la scelta del nuovo candidato da presentare per le prossime elezioni.
[suona il citofono]
[musica elettronica]
[gemiti]
[gemiti]
[sniffa]
[gemiti]
[gemiti]
Ah!
[gemiti]
[ansima]
Che succede?
[voci indistinte]
[ansima]
Porca di quella puttana, ma ce li hai gli occhi?
Lo vedi o no la gente che locali vuole frequentare?
Su quel chiosco, ci butti due spicci e si riempie.
Un certo tipo di clientela, oltre a mangiare e bere, vuole pippare.
Livia, senti questa che fa ridere! Questo vuole 

In [ ]:
line = '<p begin="496800000t" end="518800000t" region="region_1" xml:id="subtitle0"><span style="style_2">[radio] </span><span style="style_1">Oggi, dopo sette anni di mandato,</span></p>'
xml_cleaner(line)
#to_rep = line[0:98]
#line.replace(to_rep, ' ').strip()